# லேப்-2-அறிமுகம் பை-3.5 விசன்


In [ ]:
! pip install opencv-python

In [ ]:
import cv2
import numpy as np

In [ ]:
def save_keyframes(video_path, output_folder):
    videoCapture = cv2.VideoCapture(video_path)
    success, frame = videoCapture.read()
    i = 0
    while success:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        hist = cv2.calcHist([gray_frame], [0], None, [256], [0, 256])
        
        success, next_frame = videoCapture.read()
        if not success:
            break
        
        next_gray_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
        
        next_hist = cv2.calcHist([next_gray_frame], [0], None, [256], [0, 256])
        
        similarity = cv2.compareHist(hist, next_hist, cv2.HISTCMP_CORREL)
        
        if similarity < 0.9:
            i += 1
            cv2.imwrite(f"{output_folder}/keyframe_{i}.jpg", frame)
            print(f"Saved keyframe {i}")
        
        frame = next_frame

    videoCapture.release()

In [ ]:
save_keyframes('../examples/video/car.mp4', './output')

In [ ]:
from PIL import Image
import requests, base64

In [ ]:
images = [] 
placeholder = "" 
for i in range(1,4): 
    with open("../examples/output/keyframe_"+str(i)+".jpg", "rb") as f:

        images.append(Image.open("../examples/output/keyframe_"+str(i)+".jpg"))
        placeholder += f"<|image_{i}|>\n"
        # print(i)

In [ ]:
from notebook_utils import device_widget

device = device_widget(default="GPU", exclude=["NPU"])

device

In [ ]:
from ov_phi3_vision import OvPhi3Vision

In [ ]:
out_dir = "../model/phi-3.5-vision-128k-instruct-ov"

In [ ]:
model = OvPhi3Vision(out_dir, device.value)

In [ ]:
from transformers import AutoProcessor, TextStreamer

messages = [
    {"role": "user", "content":  placeholder+"Summarize the video."},
]

processor = AutoProcessor.from_pretrained(out_dir, trust_remote_code=True)

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, images, return_tensors="pt")

generation_args = {"max_new_tokens": 500, "do_sample": False, "streamer": TextStreamer(processor.tokenizer, skip_prompt=True, skip_special_tokens=True)}

print("Summary:")
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
பொறுப்பு விலக்கு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவையான [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டது. நாங்கள் துல்லியத்திற்காக முயற்சி செய்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருப்பதற்கு வாய்ப்பு உள்ளது என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அசல் ஆவணம் அதன் சொந்த மொழியில் அதிகாரப்பூர்வ மூலமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, மனிதப் பயின்று தொழில்முறை மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பின் பயன்பாட்டினால் ஏற்படும் எந்தவொரு தவறான புரிதலுக்கும் அல்லது தவறான விளக்கத்திற்கும் நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
